In [ ]:
import boto3

region_name = boto3.Session().region_name

cfn = boto3.client('cloudformation', region_name=region_name)

def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "VectorBasedSemanticSearchStack"

outputs = get_cfn_outputs(cloudformation_stack_name)
es_host = outputs['OpenSearchDomainEndpoint']
secret_id = outputs['MasterUserSecretId']

outputs

In [ ]:
import json

secretsmanager_client = boto3.client('secretsmanager', region_name=region_name)
response = secretsmanager_client.get_secret_value(SecretId=secret_id)

secret_string = json.loads(response['SecretString'])
secret_string

In [ ]:
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub

In [ ]:
MODULE_DIR_URL = "https://tfhub.dev/google/universal-sentence-encoder/2"

In [ ]:
embed = hub.Module(MODULE_DIR_URL)
text_ph = tf.placeholder(tf.string)
embeddings = embed(text_ph)

config = tf.ConfigProto()
session = tf.Session(config=config)
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())

In [ ]:
def embed_text(text):
    vectors = session.run(embeddings, feed_dict={text_ph: text})
    return [vector.tolist() for vector in vectors]

In [ ]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch(
    [es_host],
    http_auth=(secret_string['username'], secret_string['password']),
    scheme="https",
    port=443,
)

In [ ]:
knn_index = {
  "settings": {
    "index.knn": "true"
  },
   "mappings": {
    "dynamic": "true",
    "_source": {
      "enabled": "true"
    },
    "properties": {
      "user": {
        "type": "keyword"
      },
      "creationDate": {
        "type": "date"
      },
      "title": {
        "type": "text"
      },
      "title_vector": {
        "type": "knn_vector",
        "dimension": 512
      },
      "questionId": {
        "type": "keyword"
      },
      "answerId": {
        "type": "keyword"
      },
      "acceptedAnswerId": {
        "type": "keyword"
      },
      "tags": {
        "type": "keyword"
      },
      "body": {
        "type": "text"
      },
      "type": {
        "type": "keyword"
      }
    }
  }
}

In [ ]:
INDEX_NAME = 'posts'

In [ ]:
# es_client.indices.delete(index=INDEX_NAME, ignore=[404])
es_client.indices.create(index=INDEX_NAME, body=knn_index, ignore=[400, 404])
es_client.indices.get(index=INDEX_NAME)

In [ ]:
from elasticsearch.helpers import bulk

def index_batch(docs):
    titles = [doc["title"] for doc in docs]
    title_vectors = embed_text(titles)

    requests = []
    for i, doc in enumerate(docs):
        request = doc
        request["_op_type"] = "index"
        request["_index"] = INDEX_NAME
        request["title_vector"] = title_vectors[i]
        requests.append(request)
    bulk(es_client, requests)

In [ ]:
sample_docs = [
    {"user":"11","tags":[".net","math"],"questionId":"14","creationDate":"2008-08-01T00:59:11.177","title":"Difference between Math.Floor() and Math.Truncate()","type":"question","body":"What is the difference between Math.Floor() and Math.Truncate() in .NET? "},
    {"user":"2","tags":["c#","linq","web-services",".net-3.5"],"questionId":"16","creationDate":"2008-08-01T04:59:33.643","title":"Filling a DataSet or DataTable from a LINQ query result set","acceptedAnswerId":"12446","type":"question","body":"How do you expose a LINQ query as an ASMX web service? Usually, from the business tier, I can return a typed DataSet or DataTable which can be serialized for transport over ASMX. How can I do the same for a LINQ query? Is there a way to populate a typed DataSet or DataTable via a LINQ query? public static MyDataTable CallMySproc() { string conn = \"...\"; MyDatabaseDataContext db = new MyDatabaseDataContext(conn); MyDataTable dt = new MyDataTable(); // execute a sproc via LINQ var query = from dr in db.MySproc().AsEnumerable select dr; // copy LINQ query resultset into a DataTable -this does not work ! dt = query.CopyToDataTable(); return dt; } How can I get the result set of a LINQ query into a DataSet or DataTable? Alternatively, is the LINQ query serializeable so that I can expose it as an ASMX web service? "},
    {"user":"2","tags":["mysql","database","binary-data","data-storage"],"questionId":"17","creationDate":"2008-08-01T05:09:55.993","title":"Binary Data in MySQL","acceptedAnswerId":"26","type":"question","body":"How do I store binary data in MySQL? "},
    {"user":"17766","tags":["database","database-design"],"questionId":"158392","creationDate":"2008-10-01T16:10:15.770","title":"Primary Key versus Unique Constraint?","acceptedAnswerId":"158432","type":"question","body":"I'm currently designing a brand new database. In school, we always learned to put a primary key in each table. I read a lot of articles/discussions/newsgroups posts saying that it's better to use unique constraint (aka unique index for some db) instead of PK. What's your point of view? "},
    {"user":"24179","tags":["c#","deployment","clickonce","citrix","smartclient"],"questionId":"158385","creationDate":"2008-10-01T16:09:19.187","title":"Citrix - how to keep smartclient apps from re-downloading every time they are launched","acceptedAnswerId":"166969","type":"question","body":"Our company uses Citrix to remote into a terminal server for remote users to launch smart client apps within a virtual window on their machine. The problem is that smartclient apps are being downloaded each time the user launches them eventhough the version on the remote citrix server has not change. This is due to the user's profile being purged each time they close their Citrix session. Is there any way to avoid this and still continue to purge the user's profile? Not purging the profile leads to wasted space on the citrix servers and corrupt profile issues. "},
    {"user":"9217","tags":["delphi","unicode","delphi-2009","delphi-2006"],"questionId":"158382","creationDate":"2008-10-01T16:08:34.617","title":"Open an ANSI file and Save a a Unicode file using Delphi","acceptedAnswerId":"158435","type":"question","body":"For some reason, lately the *.UDL files on many of my client systems are no longer compatible as they were once saved as ANSI files, which is no longer compatible with the expected UNICODE file format. The end result is an error dialog which states \"the file is not a valid compound file\". What is the easiest way to programatically open these files and save as a unicode file? I know I can do this by opening each one in notepad and then saving as the same file but with the \"unicode\" selected in the encoding section of the save as dialog, but I need to do this in the program to cut down on support calls. This problem is very easy to duplicate, just create a *.txt file in a directory, rename it to *.UDL, then edit it using the microsoft editor. Then open it in notepad and save as the file as an ANSI encoded file. Try to open the udl from the udl editor and it will tell you its corrupt. then save it (using notepad) as a Unicode encoded file and it will open again properly. "},
    {"user":"24200","tags":["php","apache",".htaccess","mod-rewrite"],"questionId":"158384","creationDate":"2008-10-01T16:08:52.823","title":"What happens first? .htaccess or php code?","acceptedAnswerId":"158404","type":"question","body":"If I use mod_rewrite to control all my 301 redirects, does this happen before my page is served? so if I also have a bunch of redirect rules in a php script that runs on my page, will the .htaccess kick in first? "},
    {"user":"87","tags":["c#",".net","compression","decompression"],"questionId":"145","creationDate":"2008-08-01T17:13:08.933","title":"Compressing / Decompressing Folders & Files","type":"question","body":"Does anyone know of a good way to compress or decompress files and folders in C# quickly? Handling large files might be necessary. "},
    {"user":"178","tags":["c#","tiff"],"questionId":"44476","creationDate":"2008-09-04T18:55:37.447","title":"Compressing a TIF file","acceptedAnswerId":"45131","type":"question","body":"I'm trying to convert a multipage color tiff file to a c# CompressionCCITT3 tiff in C#. I realize that I need to make sure that all pixels are 1 bit. I have not found a useful example of this online. "},
    {"user":"22410","tags":["perl","apache","mod-deflate","lwp"],"questionId":"136432","creationDate":"2008-09-25T21:56:29.443","title":"Compressing HTTP request with LWP, Apache, and mod_deflate","type":"question","body":"I have a client/server system that performs communication using XML transferred using HTTP requests and responses with the client using Perl's LWP and the server running Perl's CGI.pm through Apache. In addition the stream is encrypted using SSL with certificates for both the server and all clients. This system works well, except that periodically the client needs to send really large amounts of data. An obvious solution would be to compress the data on the client side, send it over, and decompress it on the server. Rather than implement this myself, I was hoping to use Apache's mod_deflate's \"Input Decompression\" as described here. The description warns: If you evaluate the request body yourself, don't trust the Content-Length header! The Content-Length header reflects the length of the incoming data from the client and not the byte count of the decompressed data stream. So if I provide a Content-Length value which matches the compressed data size, the data is truncated. This is because mod_deflate decompresses the stream, but CGI.pm only reads to the Content-Length limit. Alternatively, if I try to outsmart it and override the Content-Length header with the decompressed data size, LWP complains and resets the value to the compressed length, leaving me with the same problem. Finally, I attempted to hack the part of LWP which does the correction. The original code is: # Set (or override) Content-Length header my $clen = $request_headers->header('Content-Length'); if (defined($$content_ref) && length($$content_ref)) { $has_content = length($$content_ref); if (!defined($clen) || $clen ne $has_content) { if (defined $clen) { warn \"Content-Length header value was wrong, fixed\"; hlist_remove(\\@h, 'Content-Length'); } push(@h, 'Content-Length' => $has_content); } } elsif ($clen) { warn \"Content-Length set when there is no content, fixed\"; hlist_remove(\\@h, 'Content-Length'); } And I changed the push line to: push(@h, 'Content-Length' => $clen); Unfortunately this causes some problem where content (truncated or not) doesn't even get to my CGI script. Has anyone made this work? I found this which does compression on a file before uploading, but not compressing a generic request. "},
    {"user":"10059","tags":["java","ip-address","hostname"],"questionId":"66923","creationDate":"2008-09-15T21:01:02.480","title":"How do you tell whether a string is an IP or a hostname","acceptedAnswerId":"66944","type":"question","body":"So you have a String that is retrieved from an admin web UI (so it is definitely a String). How can you find out whether this string is an IP address or a hostname in Java? Update: I think I didn't make myself clear, I was more asking if there is anything in the Java SDK that I can use to distinguish between IPs and hostnames? Sorry for the confusion and thanks for everybody who took/will take the time to answer this. "},
    {"user":"13172","tags":["javascript","dns"],"questionId":"102605","creationDate":"2008-09-19T15:06:16.250","title":"Can I perform a DNS lookup (hostname to IP address) using client-side Javascript?","acceptedAnswerId":"102670","type":"question","body":"I would like to use client-side Javascript to perform a DNS lookup (hostname to IP address) as seen from the client's computer. Is that possible? "},
    {"user":"19935","tags":["windows","sysadmin"],"questionId":"110920","creationDate":"2008-09-21T12:37:59.040","title":"Changing Hostname / IP Address of Windows System Mounted as an Image","acceptedAnswerId":"111005","type":"question","body":"I'm looking for a way to change the hostname and IP address of a Windows XP system that is mounted via a loop-back image on a Linux system. So basically I have access to the Windows XP system on a file level, but I cannot execute any programs on it. A way similar to editing the /etc/hostname and whatever network configuration file under Linux. The only ways I've found so far would include running a tool after boot, e.g. MS sysprep or use a solution like Acronis Snap Deploy. "},
    {"user":"680","tags":["python","floating-point"],"questionId":"5415","creationDate":"2008-08-07T22:24:27.740","title":"Convert Bytes to Floating Point Numbers in Python","acceptedAnswerId":"73281","type":"question","body":"I have a binary file that I have to parse and I'm using Python. Is there a way to take 4 bytes and convert it to a single precision floating point number? "},
    {"user":"742","tags":["python"],"questionId":"10123","creationDate":"2008-08-13T17:46:41.940","title":"How do I treat an integer as an array of bytes in Python?","acceptedAnswerId":"10129","type":"question","body":"I'm trying to decode the result of the Python os.wait() function. This returns, according to the Python docs: a tuple containing its pid and exit status indication: a 16-bit number, whose low byte is the signal number that killed the process, and whose high byte is the exit status (if the signal number is zero); the high bit of the low byte is set if a core file was produced. How do I decode the exit status indication (which is an integer) to obtain the high and low byte? To be specific, how do I implement the decode function used in the following code snippet: (pid,status) = os.wait() (exitstatus, signum) = decode(status) "},
    {"user":"2348","tags":["c#",".net","arrays","string","truncate"],"questionId":"34395","creationDate":"2008-08-29T14:51:24.727","title":"How do I truncate a string while converting to bytes in C#?","acceptedAnswerId":"34431","type":"question","body":"I would like to put a string into a byte array, but the string may be too big to fit. In the case where it's too large, I would like to put as much of the string as possible into the array. Is there an efficient way to find out how many characters will fit? "},
    {"user":"383","tags":["c#","integer"],"questionId":"3213","creationDate":"2008-08-06T09:21:09.490","title":"Convert integers to written numbers","acceptedAnswerId":"3267","type":"question","body":"Is there an efficient method of converting an integer into the written numbers, for example: string Written = IntegerToWritten(21); would return \"Twenty One\". Is there any way of doing this that doesn't involve a massive look-up table? "},
    {"user":"6042","tags":["algorithm","math"],"questionId":"58493","creationDate":"2008-09-12T08:18:38.373","title":"Algorithm to find a common multiplier to convert decimal numbers to whole numbers","acceptedAnswerId":"58497","type":"question","body":"I have an array of numbers that potentially have up to 8 decimal places and I need to find the smallest common number I can multiply them by so that they are all whole numbers. I need this so all the original numbers can all be multiplied out to the same scale and be processed by a sealed system that will only deal with whole numbers, then I can retrieve the results and divide them by the common multiplier to get my relative results. Currently we do a few checks on the numbers and multiply by 100 or 1,000,000, but the processing done by the *sealed system can get quite expensive when dealing with large numbers so multiplying everything by a million just for the sake of it isn\u2019t really a great option. As an approximation lets say that the sealed algorithm gets 10 times more expensive every time you multiply by a factor of 10. What is the most efficient algorithm, that will also give the best possible result, to accomplish what I need and is there a mathematical name and/or formula for what I\u2019m need? *The sealed system isn\u2019t really sealed. I own/maintain the source code for it but its 100,000 odd lines of proprietary magic and it has been thoroughly bug and performance tested, altering it to deal with floats is not an option for many reasons. It is a system that creates a grid of X by Y cells, then rects that are X by Y are dropped into the grid, \u201cproprietary magic\u201d occurs and results are spat out \u2013 obviously this is an extremely simplified version of reality, but it\u2019s a good enough approximation. So far there are quiet a few good answers and I wondered how I should go about choosing the \u2018correct\u2019 one. To begin with I figured the only fair way was to create each solution and performance test it, but I later realised that pure speed wasn\u2019t the only relevant factor \u2013 an more accurate solution is also very relevant. I wrote the performance tests anyway, but currently the I\u2019m choosing the correct answer based on speed as well accuracy using a \u2018gut feel\u2019 formula. My performance tests process 1000 different sets of 100 randomly generated numbers. Each algorithm is tested using the same set of random numbers. Algorithms are written in .Net 3.5 (although thus far would be 2.0 compatible) I tried pretty hard to make the tests as fair as possible. Greg \u2013 Multiply by large number and then divide by GCD \u2013 63 milliseconds Andy \u2013 String Parsing \u2013 199 milliseconds Eric \u2013 Decimal.GetBits \u2013 160 milliseconds Eric \u2013 Binary search \u2013 32 milliseconds Ima \u2013 sorry I couldn\u2019t figure out a how to implement your solution easily in .Net (I didn\u2019t want to spend too long on it) Bill \u2013 I figure your answer was pretty close to Greg\u2019s so didn\u2019t implement it. I\u2019m sure it\u2019d be a smidge faster but potentially less accurate. So Greg\u2019s Multiply by large number and then divide by GCD\u201d solution was the second fastest algorithm and it gave the most accurate results so for now I\u2019m calling it correct. I really wanted the Decimal.GetBits solution to be the fastest, but it was very slow, I\u2019m unsure if this is due to the conversion of a Double to a Decimal or the Bit masking and shifting. There should be a similar usable solution for a straight Double using the BitConverter.GetBytes and some knowledge contained here: http://blogs.msdn.com/bclteam/archive/2007/05/29/bcl-refresher-floating-point-types-the-good-the-bad-and-the-ugly-inbar-gazit-matthew-greig.aspx but my eyes just kept glazing over every time I read that article and I eventually ran out of time to try to implement a solution. I\u2019m always open to other solutions if anyone can think of something better. "},
]

In [ ]:
index_batch(sample_docs)

In [ ]:
import time

def handle_query(client, query, limit=5):
    embedding_start = time.time()
    query_vector = embed_text([query])[0]
    embedding_time = time.time() - embedding_start

    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, doc['title_vector']) + 1.0",
                "params": {"query_vector": query_vector}
            }
        }
    }

    search_start = time.time()
    response = client.search(
        index=INDEX_NAME,
        body={
            "size": limit,
            "query": script_query,
            "_source": {"includes": ["title", "questionId"]}
        }
        #, explain=True #XXX: You can check how to calculate score
    )
    search_time = time.time() - search_start

    print()
    print("{} total hits.".format(response["hits"]["total"]["value"]))
    print("embedding time: {:.2f} ms".format(embedding_time * 1000))
    print("search time: {:.2f} ms".format(search_time * 1000))
    for hit in response["hits"]["hits"]:
        print("id: {}, score: {}".format(hit["_id"], hit["_score"]))
        print(hit["_source"])
        print()

In [ ]:
query = 'zipping up files'
handle_query(es_client, query)

In [ ]:
query = 'Filling a DataSet or DataTable from'
handle_query(es_client, query)

In [ ]:
query = 'determine if something is an IP'
handle_query(es_client, query)

In [ ]:
query = 'translate bytes to doubles'
handle_query(es_client, query)